In [1]:
using Revise

ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
using FileIO
using JLD2
using RiskSensitiveSAC

┌ Error: Unable to load the 'rospy' python package!
│ Has an environment setup script been run?
└ @ RobotOS /home/kh-ryu/.julia/packages/RobotOS/j0Tsl/src/RobotOS.jl:35


┌ Info: Number of Julia Thread(s): 1
└ @ RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3060
└ @ RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:40


┌ Info: Python executable used by PyCall: /home/kh-ryu/anaconda3/bin/python
└ @ RiskSensitiveSAC /home/kh-ryu/DRC_Project/DRC_Trajectron/src/RiskSensitiveSAC.jl:41


InitError: InitError: PyError ($(Expr(:escape, :(ccall(#= /home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'ModuleNotFoundError'>
ModuleNotFoundError("No module named 'easydict'")
  File "/home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 12, in <module>
    ns = PyDict{String,PyObject}()

during initialization of module RiskSensitiveSAC

# 1. Default

In [23]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

incl_robot_node = true;
use_robot_future = true;
sim_horizon = 4.4;
prediction_rng_seed = 123;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [24]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

Scene Mode: data
Prediction Mode: trajectron
Deterministic Prediction: false


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'NameError'>
NameError("name 'load_hyperparams' is not defined")
  File "/home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 1, in <module>
    const Py_single_input = 256  # from Python.h


In [25]:
result, controller, ado_inputs = 
evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
         target_speed, measurement_schedule, target_trajectory,
         pos_error_replan, nominal_control=false,
         ado_id_removed="PEDESTRIAN/236");

UndefVarError: UndefVarError: target_trajectory not defined

In [26]:
display_log(result.log)

UndefVarError: UndefVarError: result not defined

In [27]:
e_state = result.w_history[end].e_state;
w = result.w_history[end];
u_schedule = result.u_schedule_history[end];
target_trajectory = result.target_trajectory_history[end];

UndefVarError: UndefVarError: result not defined

In [28]:
rpf = get_robot_present_and_future(e_state, u_schedule, controller.sim_param, controller.cnt_param);

UndefVarError: UndefVarError: controller not defined

In [29]:
function get_clipped_prediction_dict(prediction_dict::Dict{String, Array{Float64, 3}},
                                     num_samples::Int64,
                                     nominal_control_idx::Int64)
    clipped_preds_dict = Dict{String, Array{Float64, 3}}();
    for key in keys(prediction_dict)
        clipped_array =
            prediction_dict[key][num_samples*(nominal_control_idx - 1) + 1:num_samples*nominal_control_idx, :, :];
        clipped_preds_dict[key] = copy(clipped_array);
    end
    return clipped_preds_dict
end

get_clipped_prediction_dict (generic function with 1 method)

In [30]:
prediction_dict = sample_future_ado_positions!(controller.predictor, ado_inputs, rpf);

UndefVarError: UndefVarError: controller not defined

In [31]:
u_nominal_idx = 12;
prediction_dict_clipped = get_clipped_prediction_dict(prediction_dict,
                                              num_samples, u_nominal_idx)
nominal_trajectory = [rpf[u_nominal_idx, t, 1:2] for t = 1:size(rpf, 2)];
plt = visualize!(Dict(), w, target_trajectory, prediction_dict_clipped, num_samples, nominal_trajectory,
         xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5)

UndefVarError: UndefVarError: prediction_dict not defined

In [32]:
u_nominal_idx = 6;
prediction_dict_clipped = get_clipped_prediction_dict(prediction_dict,
                                              num_samples, u_nominal_idx)
nominal_trajectory = [rpf[u_nominal_idx, t, 1:2] for t = 1:size(rpf, 2)];
plt = visualize!(Dict(), w, target_trajectory, prediction_dict_clipped, num_samples, nominal_trajectory,
         xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5)

UndefVarError: UndefVarError: prediction_dict not defined

# 2. Deterministic (mode-mode) Prediction

In [33]:
include("$(@__DIR__)/../scripts/default_params/params_data_trajectron.jl");

incl_robot_node = true;
use_robot_future = true;
sim_horizon = 4.4;
num_samples = 1;
deterministic = true;

include("$(@__DIR__)/../scripts/parameter_setup.jl");

In [34]:
scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed =
controller_setup(scene_param,
                 predictor_param,
                 prediction_device=prediction_device,
                 cost_param=cost_param,
                 cnt_param=cnt_param,
                 dtc=dtc,
                 sim_horizon=sim_horizon,
                 ado_id_to_replace="PEDESTRIAN/236",
                 verbose=true);

Scene Mode: data
Prediction Mode: trajectron
Deterministic Prediction: true


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'NameError'>
NameError("name 'load_hyperparams' is not defined")
  File "/home/kh-ryu/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 1, in <module>
    const Py_single_input = 256  # from Python.h


In [35]:
result, controller, ado_inputs = 
evaluate(scene_loader, controller, w_init, maximum(target_trajectory)[2],
         target_speed, measurement_schedule, target_trajectory,
         pos_error_replan, nominal_control=false,
         ado_id_removed="PEDESTRIAN/236");

UndefVarError: UndefVarError: target_trajectory not defined

In [36]:
display_log(result.log)

UndefVarError: UndefVarError: result not defined

In [37]:
e_state = result.w_history[end].e_state;
w = result.w_history[end];
u_schedule = result.u_schedule_history[end];
target_trajectory = result.target_trajectory_history[end];

UndefVarError: UndefVarError: result not defined

In [38]:
rpf = get_robot_present_and_future(e_state, u_schedule, controller.sim_param, controller.cnt_param);

UndefVarError: UndefVarError: controller not defined

In [39]:
prediction_dict = sample_future_ado_positions!(controller.predictor, ado_inputs,
                                               rpf);

UndefVarError: UndefVarError: controller not defined

In [40]:
u_nominal_idx = 12;
prediction_dict_clipped = get_clipped_prediction_dict(prediction_dict,
                                              num_samples, u_nominal_idx)
nominal_trajectory = [rpf[u_nominal_idx, t, 1:2] for t = 1:size(rpf, 2)];
plt = visualize!(Dict(), w, target_trajectory, prediction_dict_clipped, num_samples, nominal_trajectory,
         xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5)

UndefVarError: UndefVarError: prediction_dict not defined

In [41]:
u_nominal_idx = 6;
prediction_dict_clipped = get_clipped_prediction_dict(prediction_dict,
                                              num_samples, u_nominal_idx)
nominal_trajectory = [rpf[u_nominal_idx, t, 1:2] for t = 1:size(rpf, 2)];
plt = visualize!(Dict(), w, target_trajectory, prediction_dict_clipped, num_samples, nominal_trajectory,
         xlim=(-3. + -5.263534, 13. + -5.314636), 
         ylim=(0. + -5.263534, 10. + -5.314636), figsize=(600, 400), 
         legendfontsize=7, legend=:bottomright, markersize=5)

UndefVarError: UndefVarError: prediction_dict not defined